# Deduplication Example

Before running this notebook on Google Colab, please make sure to set the runtime type to "GPU". Do that by going to the "Runtime" menu and selecting "Change runtime type".

## Boilerplate

In [ ]:
!pip install git+https://github.com/egu8/entity-embed

In [ ]:
!pip install "matplotlib==3.1.1" \
             "pynndescent==0.5.2" \
             "scikit-learn==0.24.1" \
             "seaborn==0.11.1" \
             "unidecode==1.1.2"

In [ ]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [ ]:
import sys

sys.path.insert(0, '..')

In [ ]:
import entity_embed

In [ ]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

## Load Dataset

Click this link [here](https://drive.google.com/drive/folders/1OhKJ413oKe9zdGy3wSaMteMrqFqvGArZ?usp=sharing) to get access to the Google Drive which is hosting the datasets.

Download the zipped datasets and upload it the runtime files for this Colab.

Note that it may take a few seconds to upload the zipped file due to the size.



In [ ]:
!unzip bookcover.zip
!unzip book-covers.zip

In [ ]:
import os

DATAFILE = "bookcover.csv"

Now we must read the CSV dataset into a `dict` called `record_dict`.

`record_dict` will contain all records from the dataset, and each record will have the indication of the true cluster it belongs to in the field `CID`.

So `CID` is our `cluster_field`. Entity Embed needs that to train, validate, and test.

We'll dynamically attribute an `id` to each record using `enumerate`. Entity Embed needs that too.

In [ ]:
import csv

record_dict = {}
cluster_field = 'CID'

with open(DATAFILE, newline='') as f:
    for current_record_id, record in enumerate(csv.DictReader(f)):
        record['id'] = current_record_id
        record[cluster_field] = int(record[cluster_field])  # convert cluster_field to int
        record_dict[current_record_id] = record

Here's an example of a record:

In [ ]:
record_dict[83]

OrderedDict([('', '83'),
             ('image',
              'https://d1w7fb2mkkr3kw.cloudfront.net/assets/images/book/mid/9781/1019/9781101980385.jpg'),
             ('name', "It Didn't Start with You"),
             ('author', 'Mark Wolynn'),
             ('format', 'Paperback'),
             ('book_depository_stars', '3.5'),
             ('price', '13.18'),
             ('currency', '$'),
             ('old_price', '16'),
             ('isbn', '9781101980385'),
             ('category', 'Medical'),
             ('img_paths', 'dataset/Medical/0000084.jpg'),
             ('TID', '84'),
             ('CID', 84),
             ('CTID', '1'),
             ('id', 83)])

How many clusters this dataset has?

In [ ]:
cluster_total = len(set(record[cluster_field] for record in record_dict.values()))
cluster_total

22708

From all clusters, we'll use only 20% for training, and other 20% for validation to test how well we can generalize:

In [ ]:
from entity_embed.data_utils import utils

train_record_dict, valid_record_dict, test_record_dict = utils.split_record_dict_on_clusters(
    record_dict=record_dict,
    cluster_field=cluster_field,
    train_proportion=0.2,
    valid_proportion=0.2,
    random_seed=random_seed)

21:09:55 INFO:Singleton cluster sizes (train, valid, test):(1167, 1167, 3501)
21:09:55 INFO:Plural cluster sizes (train, valid, test):(3374, 3374, 10125)


Note we're splitting the data on **clusters**, not records, so the record counts vary:

In [ ]:
len(train_record_dict), len(valid_record_dict), len(test_record_dict)

(13006, 13080, 39076)

## Preprocess

We'll perform a very minimal preprocessing of the dataset. We want to simply force ASCII chars, lowercase all chars, and strip leading and trailing whitespace.

The fields we'll clean are the ones we'll use:

In [ ]:
field_list = ['name', 'author', 'format', 'price', 'category', 'book_depository_stars']

In [ ]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record in record_dict.values():
    for field in field_list:
        record[field] = clean_str(record[field])

In [ ]:
utils.subdict(record_dict[83], field_list)

{'author': 'mark wolynn',
 'book_depository_stars': '3.5',
 'category': 'medical',
 'format': 'paperback',
 'name': "it didn't start with you",
 'price': '13.18'}

Forcing ASCII chars in this dataset is useful to improve recall because there's little difference between accented and not-accented chars here. Also, this dataset contains mostly latin chars.

## Configure Entity Embed fields

Now we will define how record fields will be numericalized and encoded by the neural network. First we set an `alphabet`, here we'll use ASCII numbers, letters, symbols and space:

In [ ]:
from entity_embed.data_utils.field_config_parser import DEFAULT_ALPHABET

alphabet = DEFAULT_ALPHABET
''.join(alphabet)

'0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '

It's worth noting you can use any alphabet you need, so the accent removal we performed is optional.

Then we set an `field_config_dict`. It defines `field_type`s that determine how fields are processed in the neural network:

In [ ]:
field_config_dict = {
    'author': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'author_semantic': {
        'key': 'author',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'name_semantic': {
        'key': 'name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'book_depository_stars': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'category': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'format': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

Then we use our `field_config_dict` to get a `record_numericalizer`. This object will convert the strings from our records into tensors for the neural network.

The same `record_numericalizer` must be used on ALL data: train, valid, test. This ensures numericalization will be consistent. Therefore, we pass `record_list=record_dict.values()`:

In [ ]:
from entity_embed import FieldConfigDictParser

record_numericalizer = FieldConfigDictParser.from_dict(field_config_dict, record_list=record_dict.values())

21:10:15 INFO:For field=author, computing actual max_str_len
21:10:15 INFO:actual_max_str_len=17 must be even to enable NN pooling. Updating to 18
21:10:15 INFO:For field=author, using actual_max_str_len=18
21:10:23 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
21:11:20 INFO:For field=name, computing actual max_str_len
21:11:21 INFO:For field=name, using actual_max_str_len=20
21:11:21 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
21:11:32 INFO:For field=book_depository_stars, computing actual max_str_len
21:11:32 INFO:actual_max_str_len=3 must be even to enable NN pooling. Updating to 4
21:11:32 INFO:For field=book_depository_stars, using actual_max_str_len=4
21:11:32 INFO:For field=category, computing actual max_str_len
21:11:32 INFO:For field=category, using actual_max_str_len=30
21:11:32 INFO:For field=format, computing actual max_str_len
21:11:32 INFO:For field=format, using actual_max_str_len=22
21:11:32 INFO:For field=price, computing actual max_str_len
21:11:32

## Initialize Data Module

under the hood, Entity Embed uses [pytorch-lightning](https://pytorch-lightning.readthedocs.io/en/latest/), so we need to create a datamodule object:

In [ ]:
from entity_embed import DeduplicationDataModule

batch_size = 32
eval_batch_size = 64
datamodule = DeduplicationDataModule(
    train_record_dict=train_record_dict,
    valid_record_dict=valid_record_dict,
    test_record_dict=test_record_dict,
    cluster_field=cluster_field,
    record_numericalizer=record_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

We've used `DeduplicationDataModule` because we're doing Deduplication of a single dataset/table (a.k.a. Entity Clustering, Entity Resolution, etc.).

We're NOT doing Record Linkage of two datasets here. Check the other notebook [Record-Linkage-Example](./Record-Linkage-Example.ipynb) if you want to learn how to do it with Entity Embed.

## Training

Now the training process! Thanks to pytorch-lightning, it's easy to train, validate, and test with the same datamodule.

We must choose the K of the Approximate Nearest Neighbors, i.e., the top K neighbors our model will use to find duplicates in the embedding space. Below we're setting it on `ann_k` and initializing the `EntityEmbed` model object:

In [ ]:
from entity_embed import EntityEmbed

ann_k = 100
model = EntityEmbed(
    record_numericalizer,
    ann_k=ann_k,
)

To train, Entity Embed uses [pytorch-lightning Trainer](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html) on it's `EntityEmbed.fit` method.

Since Entity Embed is focused in recall, we'll use `valid_recall_at_0.3` for early stopping. But we'll set `min_epochs = 5` to avoid a very low precision.

`0.3` here is the threshold for **cosine similarity of embedding vectors**, so possible values are between -1 and 1. We're using a validation metric, and the training process will run validation on every epoch end due to `check_val_every_n_epoch=1`.

We also set `tb_name` and `tb_save_dir` to use Tensorboard. Run `tensorboard --logdir notebooks/tb_logs` to check the train and valid metrics during and after training.

In [ ]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_recall_at_0.3",
    tb_save_dir='tb_logs',
    tb_name='music',
)

21:12:07 INFO:GPU available: True, used: True
21:12:07 INFO:TPU available: False, using: 0 TPU cores
21:12:07 INFO:Train positive pair count: 20718
21:12:07 INFO:Valid positive pair count: 21040
21:12:07 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
21:14:22 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 17.4 M
1 | loss_fn     | SupConLoss | 0     
-------------------------------------------
10.2 M    Trainable params
7.2 M     Non-trainable params
17.4 M    Total params
69.706    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

22:20:16 INFO:Loading the best validation model from tb_logs/music/version_1/checkpoints/epoch=4-step=2137.ckpt...


`EntityEmbed.fit` keeps only the weights of the best validation model. With them, we can check the best performance on validation set:

In [ ]:
model.validate(datamodule)

{'valid_f1_at_0.3': 0.09007755489101,
 'valid_f1_at_0.5': 0.6094576001158665,
 'valid_f1_at_0.7': 0.8467823077232665,
 'valid_pair_entity_ratio_at_0.3': 34.10649847094801,
 'valid_pair_entity_ratio_at_0.5': 3.6701070336391437,
 'valid_pair_entity_ratio_at_0.7': 2.1906727828746178,
 'valid_precision_at_0.3': 0.047162938537993734,
 'valid_precision_at_0.5': 0.4382876783668368,
 'valid_precision_at_0.7': 0.7342779367627557,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 1.0,
 'valid_recall_at_0.7': 1.0}

And we can check which fields are most important for the final embedding:

In [ ]:
model.get_pool_weights()

{'author': 0.1505611687898636,
 'author_semantic': 0.15056373178958893,
 'book_depository_stars': 0.04295147582888603,
 'category': 0.023573685437440872,
 'format': 0.03811141103506088,
 'name': 0.2275567352771759,
 'name_semantic': 0.26123055815696716,
 'price': 0.10545119643211365}

## Testing

Again with the best validation model, we can check the performance on the test set:

In [ ]:
model.test(datamodule)

22:39:56 INFO:Test positive pair count: 62516


{'test_f1_at_0.3': 0.054005210816937224,
 'test_f1_at_0.5': 0.3539135652902481,
 'test_f1_at_0.7': 0.6562326142864641,
 'test_pair_entity_ratio_at_0.3': 57.648377520728836,
 'test_pair_entity_ratio_at_0.5': 7.441089159586447,
 'test_pair_entity_ratio_at_0.7': 3.2760262053434333,
 'test_precision_at_0.3': 0.027751981206285168,
 'test_precision_at_0.5': 0.2150030264678369,
 'test_precision_at_0.7': 0.48835283640851784,
 'test_recall_at_0.3': 1.0,
 'test_recall_at_0.5': 1.0,
 'test_recall_at_0.7': 1.0}

Entity Embed achieves Recall of ~0.99 with Pair-Entity ratio below 100 on a variety of datasets. **Entity Embed aims for high recall at the expense of precision. Therefore, this library is suited for the Blocking/Indexing stage of an Entity Resolution pipeline.**  A scalabale and noise-tolerant Blocking procedure is often the main bottleneck for performance and quality on Entity Resolution pipelines, so this library aims to solve that. Note the ANN search on embedded records returns several candidate pairs that must be filtered to find the best matching pairs, possibly with a pairwise classifier. See the [Record-Linkage-Example](./Record-Linkage-Example.ipynb) for an example of matching.